## 1. Understanding word2vec

Let's have a quick refresher on the word2vec algorithm. The key insight behind word2vec is that *'a word is known by the company it keeps'*. Concretely, suppose we have a 'center' word c and a contextual windown surrouding c. We shall refer to words that lie in this contextual window as 'outside words'. For example, in Figure 1 we see that the center word c is 'banking'. Since the context window size is 2, the outside words are 'turning', 'into', 'crises', and 'as'.

The goal of the skip-gram word2vec algorithm is to accurately learn the probability distribution P(O|C). Given a specific word o and a specific word c, we want to calculate P(O=o|C=c), which is th probability that word o is an 'outside' word for c, i.e., the probability that o falls within the contextual window of c.  

In word2vec, the conditional probability distribution is given by taking vector dot-products and applying the softmax function:
$$
\begin{equation}
    P(O=o|C=c) = \frac{\exp(u_o^T v_c)}{\sum_{w \in Vocab} \exp(u_w^Tv_c)}  \tag{1}
\label{eq:sample}
\end{equation}
$$

Here, $u_o$ is the 'outside' vector representing outside word o, and $v_c$ is the 'center' vector representing center word c. To contain these parameters, we have two matrices, $U$ and $V$. The columns of $U$ are all the 'outside' vectors $u_w$. The columns of $V$ are all of the 'center' vectors $v_w$. Both $U$ and $V$ contain a vector for every $w \in \text{Vocabulary}$.

Recall from lectures that, for a single pair of words c and o, the loss is given by:
$$
J_{naive-softmax}(v_c, o, U) = -\log P(O=o | C=c) \tag{2}
$$

Another way to view this loss is as the cross-entropy between the true distribution $y$ and the predicted distribution $\hat{y}$. Here, both $y$ and $\hat{y}$ are vectors with length equal to the number of words in the vocabulary. Furthermore, the $k^{th}$ entry in these vectors indicates the condition probabiility of the $k^{th}$ word being an 'outside word' for the given c. The true empirical distribution $y$ is a one-hot vector with a 1 for the true outside word o, and 0 everywhere else. The predicted distribution $\hat{y}$ is the probability $P(O|C=c)$ given by our model in equation(1) .

**(a)** Show that the naive-softmax given in Equation (2) is the same as cross-entropy loss between $y$ and $\hat{y}$; i.e., show that
$$
- \sum_{w \in \text{Vocab}} y_w \log(\hat{y_w}) = -\log(\hat{y_o})
$$
Your answer should be one line.

**Solution**:    
$$ - \sum_{w \in \text{Vocab}} y_w \log(\hat{y_w}) = -(y_1 \log(\hat{y_1}) + y_2 \log(\hat{y_2}) + ... + y_k \log(\hat{y_k}) + ... + y_n \log(\hat{y_n})) = -y_k \log(\hat{y_k})=   -\log(\hat{y_o}) $$

----
**(b)** Compute the partial derivative of $J_{native-softmax}(v_c, o, U)$ with respect to $v_c$. Please write your answer in terms of $y, \hat{y}$, and $U$.

**Solution**:   
[*Note*:Derivatives represent in shape convention of lecture3, shape: y ==> 1xV, U ==> dxV   ]


$$
\begin{aligned}
J_{native-softmax}(v_c, o, U)
& = - \log P(O=o|C=c) \\
& = - \log \frac{\exp(u_o^Tv_c)}{\sum_{w \in Vocab} \exp(u_w^Tv_c)} \\ 
& = - u_o^Tv_c + \log \sum_{w \in  Vocab} \exp(u_w^Tv_c) \\
\end{aligned} \tag{1}
$$

$$
\frac{\partial u_o^T v_c}{\partial v_c} = u^T \tag{2}
$$

$$
\begin{aligned}
\frac{\partial \log \sum_{w \in Vocab} \exp(u_w^T v_c)} {\partial v_c}
& = \frac{1}{\sum_{w \in Vocab} \exp(u_w^T v_c)} \frac{\partial \sum_{w \in Vocab} \exp(u_w^T v_c)}{\partial v_c}\\
& = \frac{1}{\sum_{w \in Vocab} \exp(u_w^T v_c)} \frac{\partial \sum_{w \in Vocab} \exp(u_w^T v_c)}{\partial v_c}\\
& = \frac{1}{\sum_{w \in Vocab} \exp(u_w^T v_c)} \sum_{w \in Vocab} \frac{\exp(u_w^T v_c)} {\partial v_c} \\
& = \frac{1}{\sum_{w \in Vocab} \exp(u_w^T v_c)} \sum_{w \in Vocab} \exp(u_w^T v_c)\frac{\partial u_w^T v_c} {\partial v_c} \\
& = \frac{1}{\sum_{w \in Vocab} \exp(u_w^T v_c)} \sum_{w \in Vocab} \exp(u_w^T v_c)u_w^T\\
& = \sum_{w \in Vocab} \frac{\exp(u_w^T v_c)}{\sum_{x \in Vocab} \exp(u_x^T v_c)} u_w^T
\end{aligned} \tag{3}
$$

$$
\begin{aligned}
\frac{\partial J_{native-softmax}(v_c, o, U)}{\partial v_c} 
& = \frac{\partial - u_o^Tv_c + \log \sum_{w \in  Vocab} \exp(u_w^Tv_c)}{\partial v_c} \\
& = -u_o^T + \sum_{w \in Vocab} \frac{\exp(u_w^T v_c)}{\sum_{x \in Vocab} \exp(u_x^T v_c)} u_w^T \\
& = -u_o^T + \sum_{w \in Vocab} \hat{y_w} u_w^T \\
& = (\hat{y} -y)U^T
\end{aligned}\tag{4}
$$

----
**(c)** Compute the partial derivatives of $J_{naive-softmax}(v_c, o, U)$ with respect to each of the 'outside' word vector, $u_w$'s. There will be two cases: when w=o, the true 'outside' word vector, and $w\neq o$, for all other words. Please write you answer in terms of $y, \hat{y}$, and $v_c$.

**Solution**:
$$
J_{native-softmax}(v_c, o, U) = - u_o^Tv_c + \log \sum_{w \in  Vocab} \exp(u_w^Tv_c) \tag{1}
$$
+ case (1) $w \neq o$:
$$
\begin{aligned}
\frac{\partial J_{native-softmax}(v_c, o, U)}{\partial u_w} 
& = \frac{\partial - u_o^Tv_c + \log \sum_{w \in  Vocab} \exp(u_w^Tv_c)} {\partial u_w}  \\
& = \frac{\partial \log \sum_{w \in  Vocab} \exp(u_w^Tv_c)}{\partial u_w} \\
& = \frac{1}{\sum_{w \in  Vocab} \exp(u_w^Tv_c)} \sum_{w \in  Vocab} \exp(u_w^Tv_c)\frac{\partial u_w^T v_c}{\partial u_w} \\
& = \frac{1}{\sum_{w \in  Vocab} \exp(u_w^Tv_c)} \sum_{w \in  Vocab} \exp(u_w^Tv_c)v_c^T \\
& = \sum_{w \in  Vocab} \frac{\exp(u_w^Tv_c)}{\sum_{x \in  Vocab} \exp(u_x^Tv_c)} v_c^T \\
& = \hat{y}v_c^T
\end{aligned}\tag{2}
$$

+ case (2) $w = o$:

$$
\begin{aligned}
\frac{\partial J_{native-softmax}(v_c, o, U)}{\partial u_w} 
& = \frac{\partial - u_o^Tv_c + \log \sum_{w \in  Vocab} \exp(u_w^Tv_c)} {\partial u_w}  \\
& = - \frac{\partial u_o^Tv_c}{u_w} + \frac{\partial \log \sum_{w \in  Vocab} \exp(u_w^Tv_c)}{\partial u_w} \\
& = -v_c^T +  \sum_{w \in  Vocab} \frac{\exp(u_w^Tv_c)}{\sum_{x \in  Vocab} \exp(u_x^Tv_c)} v_c^T \\
& = -v_c^T + \hat{y} v_c^T \\
\end{aligned}\tag{3}
$$

+ merge (2) and (3):

$$
\frac{\partial J_{native-softmax}(v_c, o, U)}{\partial u_w}= (\hat{y} - y)v_c^T
$$

----
**(d)** The sigmoid function is given by Equation 4:
    $$
    \delta(x) = \frac{1}{1+e^{-x}} = \frac{e^x}{e^x + 1}
    $$
    Please compute the derivative of $\delta(x)$ with respect to $x$, where $x$ is a vector.

**Solution**:
$$
\begin{aligned}
\frac{\partial \delta(x)}{\partial x}
& = \frac{\partial \frac{e^x}{e^x + 1}}{\partial x} \\
& = \frac{e^x(e^x +1) - e^xe^x}{(e^x + 1)^2} \\
& = \frac{e^x}{(e^x + 1)(e^x +1)} \\
& = \delta(x)(1 - \delta(x))
\end{aligned}
$$

----
**(e)** Now we shall consider th Negtive Sampling loss, which is an alternative to the Naive Softmax loss. Assume that K negative samples (words) are drawn from the vocabulary. For simplicity of notation we shall refer to them as $w_1, w_2, ..., w_k$ and their outside vectors as $u_1, ..., u_k$. Note that $o \notin {w_1,...w_k}$. For a center word c and an outside word o, the negative sampling loss function is given by:
$$
J_{neg-sample(v_c, o, U)} = -\log(\delta(u_o^T v_c))-\sum_{k=1}^K \log(\delta(-u_k^Tv_c))
$$
for a sample $w_1, ... w_K$, where $\delta(\cdot)$ is the sigmoid function.

Please repeat parts(b) and (c), computing the partial derivatives of $J_{neg-sample}$ with respect $v_c$, with respect to $u_o$, and with respect to a negative sample $u_k$. Please write your answers in terms of the vectors $u_o, v_c$ and $u_k$, where $k \in [1, K]$. After you've done this, describe with one sentence why this loss function is much more efficient to compute that the naive-softmax loss. Note, you should be able to use your solution to part(d) to help compute the necessary gradients here.

**Solution**:
+ (1) Partial derivatives of $J_{neg-sample}$ with respect $v_c$:

$$
\begin{aligned}
\frac{\partial J_{neg-sample(v_c, o, U)}}{\partial v_c} 
& = \frac{\partial -\log(\delta(u_o^T v_c))- \sum_{k=1}^K \log(\delta(-u_k^T v_c))}{\partial v_c} \\
& = - \frac{\partial \log(\delta(u_o^T v_c))}{ \partial v_c}  - \frac{\partial \sum_{k=1}^K \log(\delta(-u_k^T v_c))}{\partial v_c}\\
& = - \frac{1}{\delta(u_o^T v_c)}\delta(u_o^T v_c)(1-\delta(u_o^T v_c))\frac{\partial u_o^T v_c}{\partial v_c} - \sum_{k=1}^K \frac{1}{\delta(-u_k^T v_c)}\delta(-u_k^T v_c)(1 - \delta(-u_k^T v_c))\frac{\partial (-u_k^T v_c)}{\partial v_c} \\
& = (\delta(u_o^T v_c) - 1)u_o^T  - \sum_{k=1}^K(1 - \delta(-u_k^T v_c))(-u_k^T) \\
& = (\delta(u_o^T v_c) - 1)u_o^T  + \sum_{k=1}^K \delta(u_k^T v_c)u_k^T 
\end{aligned} 
$$

+ (2) Partial derivatives of $J_{neg-sample}$ with respect $u_o$:
$$
\begin{aligned}
\frac{\partial J_{neg-sample(v_c, o, U)}}{\partial u_o} 
& = \frac{\partial -\log(\delta(u_o^T v_c))- \sum_{k=1}^K \log(\delta(-u_k^T v_c))}{\partial u_o} \\ 
& = \frac{\partial -\log(\delta(u_o^T v_c))}{\partial u_o}  - \frac{\partial \sum_{k=1}^K \log(\delta(-u_k^T v_c))}{\partial u_o}  \\
& = (\delta(u_o^Tv_c) - 1) v_c^T
\end{aligned} 
$$

+ (3) Partial derivatives of $J_{neg-sample}$ with respect $u_k$:
$$
\begin{aligned}
\frac{\partial J_{neg-sample(v_c, o, U)}}{\partial u_k} 
& = \frac{\partial -\log(\delta(u_o^T v_c))- \sum_{k=1}^K \log(\delta(-u_k^T v_c))}{\partial u_k} \\ 
& = \frac{\partial -\log(\delta(u_o^T v_c))}{\partial u_k}  - \frac{\partial \sum_{k=1}^K \log(\delta(-u_k^T v_c))}{\partial u_k}  \\
& = \sum_{k=1}^K \delta(u_k^Tv_c)v_c^T
\end{aligned} 
$$

+ (4) why more efficient? Because negative sampling training neural network only updates the true outside vector and negtive samples's, much less than Vocabulary * dimension. 

----
**(f)** Suppose the center word is $c=w_t$ and the context window is $[w_{t-m}, ..., w_{t-1}, w_t, w_{t+1}, ..., w_{t+m}]$, where m is the context window size. Recall that for the skip-gram version of word2vec, the total loss for the context window is:
$$
J_{skip-gram}(v_c, w_{t-m}, ... w_{t+m}, U) = \sum_{-m \leq j \leq m, j\neq 0} J(v_c, w_{t+j}, U) \tag{6}
$$

Here $J(v_c, w_{t+j}, U)$ represents an arbitary loss term for the center word $c=w_t$ and outside word $w_{t+j}$.  

$J(v_c, w_{t+j}, U)$ could be $J_{naive-softmax}(v_c, w_{t+j}, U)$ or $J_{neg-sample}(v_c, w_{t+j}, U)$, depending on your implementation.

Write down three partial derivatives:   
(i)  $\partial J_{skip-gram}(v_c, w_{t-m}, ..., w_{t+m}, U) / \partial U$   
(ii) $\partial J_{skip-gram}(v_c, w_{t-m}, ..., w_{t+m}, U) / \partial v_c$   
(iii) $\partial J_{skip-gram}(v_c, w_{t-m}, ..., w_{t+m}, U) / \partial v_w $ when $w \neq c$.

Write your answers in terms of  $\partial J(v_c, w_{t+j}, U) / \partial U$ and $\partial J(v_c, w_{t+j}, U) / \partial v_c$. This is very simle each solution should be one line.

**Solution**:
+ (i) 
$
\partial J_{skip-gram}(v_c, w_{t-m}, ...,w_{t+m}, U)/\partial U = \sum_{-m \leq j \leq m, j\neq 0} \partial J(v_c, w_{t+j}, U)/\partial U  \\
$

+ (ii) $ \partial J_{skip-gram}(v_c, w_{t-m}, ..., w_{t+m}, U) / \partial v_c = \sum_{-m \leq j \leq m, j\neq 0} \partial J(v_c, w_{t+j}, U)/\partial v_c  $

+ (ii) $ \partial J_{skip-gram}(v_c, w_{t-m}, ..., w_{t+m}, U) / \partial v_w = O$